In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel, RFECV
from sklearn.experimental import enable_halving_search_cv # explicitly require this experimental feature
from sklearn.model_selection import KFold, HalvingGridSearchCV, train_test_split, ShuffleSplit
from sklearn.svm import LinearSVC, SVC

# Reading Features

In [2]:
# Read EEG Features
features = pd.read_csv('all_features.csv', index_col=0)
features = features.dropna(subset=['targets'])
scores_max, scores_min = max(features['targets']), min(features['targets'])
N_CLASSES = 2
for subject_session in features.index:
    score = features.loc[subject_session]['targets']
    score = int((score-1e-6 - scores_min) / (scores_max - scores_min) * N_CLASSES)  # discretise
    features.loc[subject_session, 'targets'] = score
features.loc[:, 'targets'] = features['targets'].astype(int)

# Normalize each feature column
for column in features.columns[:-1]:
    features.loc[:, column] = (features[column] - features[column].min()) / (features[column].max() - features[column].min())

In [3]:
# Remove columns completely filled with NaNs
features = features.dropna(axis=1, how='all')

### Keep only some features?

From file? 

In [4]:
#selected_features = np.loadtxt('results/optimal_pli_only.txt', dtype=str)
selected_features = np.concatenate((np.loadtxt('results/optimal_pli_only.txt', dtype=str), np.loadtxt('results/optimal_hjorth_only.txt', dtype=str)))
selected_features

Select immediately? (e.g. to make feature selection on this subset)

In [5]:
features = features[np.append(selected_features, ['targets', ])]  # keep only the selected features and the targets
features.dropna(axis=0, inplace=True)  # Discard subjects with NaNs

Just some groups?

In [6]:
#targets = features.loc[:, 'targets']

# Only Spectral
#features = features.iloc[:, :600]

# Only Hjorth
#features = features.iloc[:, 600:666]

# Only PLI
#features = features.iloc[:, 666:-1]


###########################
#features.loc[:,'targets'] = targets  # restore targets
#features = features.dropna()  # remove NaNs

Show

In [7]:
features.head()

# Model

In [8]:
params = {'class_weight': None,
          'criterion': 'gini',
          'max_depth': 5,
          'max_features': 'sqrt',
          'max_samples': None,
          'min_samples_leaf': 0.51,
          'min_samples_split': 5,
          'n_estimators': 500,
          'random_state': 0}
#model = RandomForestClassifier(**params)
model = RandomForestClassifier(500, max_depth=5, random_state=0) #, oob_score=True, warm_start=True)
#model = GradientBoostingClassifier(loss='exponential', n_estimators=100, criterion='friedman_mse', max_depth=3, learning_rate=0.1, random_state=0)
#model = SVC(kernel='rbf', C=4, class_weight='balanced', random_state=0)

# Run Purpose

Cross-Validation

In [9]:
CV = True
# 10-fold cross-validation
#cv = StratifiedShuffleSplit(n_splits=10, random_state=0)
#cv = StratifiedKFold(n_splits=10)  # sem shuffle
#cv = ShuffleSplit(n_splits=10, random_state=0)  # sem estratificação
#cv = KFold(n_splits=5)  # sem shuffle nem estratificação
# Leave-4-out cross-validation
#cv = StratifiedShuffleSplit(n_splits=179-4, random_state=0)
#cv = ShuffleSplit(n_splits=179-4, random_state=0)  # sem estratificação
#cv = KFold(n_splits=176-4)#len(features)-4)  # sem shuffle nem estratificação
cv = KFold(n_splits=len(features) // 4)  # sem shuffle nem estratificação
# My cross-validation
#cv = StratifiedLeavePOut(C=2, P=4, with_repetition=False)

In [10]:
# Sort subjects?
features = features.sort_index()
# Remove subjects without session pair?
features = features.drop(labels=['005_2', '102_2', '109_1', '195_2', '303_2'])
features.dropna(axis=0, inplace=True)  # Discard subjects with NaNs
features.head()

Selection

In [11]:
SELECTION = True
selector = RFECV
min_features_to_select = 10
max_features_to_select = 24
n_features_to_select = 24

Hyperparameters Tuning

In [12]:
HYPERPARAMETERS_TUNING = False
tunner = HalvingGridSearchCV
hyperparameters = (
    {'n_estimators': [100, 500, 1000],
     'criterion': ['gini', 'entropy', 'log_loss'],
     'max_depth': [5, 10, 20, None],
     'min_samples_split': [5, 10, 0.51],
     'min_samples_leaf': [1, 2, 0.51],
     'max_features': ['sqrt', 'log2'],
     'class_weight': [None, 'balanced', 'balanced_subsample'],
     'max_samples': [None, 0.5, 0.75, 0.9],
     }
)  # n_candidates: 7776

Sequential Selection

In [13]:
SEQUENTIAL_SELECTION = False
additional_features = None  # pli_features

# Evaluation

In [14]:
if SELECTION and CV and not HYPERPARAMETERS_TUNING:
    selector_cv = selector(estimator=model, step=1, cv=cv,
                            scoring='f1_weighted', min_features_to_select=min_features_to_select,
                            verbose=2, n_jobs=-1)

    selector_cv.fit(features.iloc[:, :-1], features['targets'])
    print(f"Optimal number of features: {selector_cv.n_features_}")

    # Plot number of features VS. cross-validation scores
    n_scores = len(selector_cv.cv_results_["mean_test_score"])
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Mean test accuracy")
    plt.scatter(
        range(min_features_to_select, n_scores + min_features_to_select),
        selector_cv.cv_results_["mean_test_score"],
        #yerr=selector_cv.cv_results_["std_test_score"],
    )
    plt.title("Recursive Feature Elimination \nwith correlated features")
    plt.show()

    # Get optimal features
    selected_features = selector_cv.get_feature_names_out()

In [15]:
if SELECTION and not CV and not HYPERPARAMETERS_TUNING:
    #selector = RFE(estimator=model, n_features_to_select=100, step=1, verbose=2)
    #selector = selector(estimator=model, max_features=max_features_to_select)
    selector = selector(LinearSVC(dual=True, penalty="l2", loss='hinge'), max_features=max_features_to_select)
    selector.fit(features.iloc[:, :-1], features['targets'])
    selected_features = selector.get_feature_names_out()

In [16]:
selected_features

In [17]:
features = features[np.append(selected_features, ['targets', ])]  # keep only the selected features and the targets
features.dropna(axis=0, inplace=True)  # Discard subjects with NaNs

In [18]:
if CV and not SELECTION and not HYPERPARAMETERS_TUNING:
    from sklearn.model_selection import cross_val_score
    #print("Cross-Validation with StratifiedLeavePOut")
    #print("Number of splits:", cv.get_n_splits(features['targets']))
    #print("Size of test sets", cv.p * cv.c * 2)
    scores = cross_val_score(model, features[selected_features], features['targets'],
                              cv=cv, scoring='f1_weighted', verbose=2, n_jobs=-1)
    print("Cross-Validation mean score:", scores.mean())
    print("Cross-Validation std score:", scores.std())
    print("Cross-Validation max score:", scores.max())
    print("Cross-Validation min score:", scores.min())

In [19]:
if not CV and not SELECTION and not HYPERPARAMETERS_TUNING and not SEQUENTIAL_SELECTION:
    # Split subjects into train and test (using sklearn)
    train_size = 0.98
    n_train = int(len(features) * train_size)
    n_test = len(features) - n_train
    train_dataset, test_dataset = train_test_split(features, train_size=train_size,
                                                   shuffle=True,
                                                   #stratify=features['targets'],
                                                   random_state=1)
    train_features = train_dataset[selected_features]
    train_targets = train_dataset['targets']
    test_features = test_dataset[selected_features]
    test_targets = test_dataset['targets']
    print("Train features shape:", train_features.shape)
    print("Test features shape:", test_features.shape)

    # Train
    model = model.fit(train_features, train_targets)

    #test_features = train_features  # FIXME: remove this line
    #test_targets = train_targets  # FIXME: remove this line

    # Test
    predictions = model.predict(test_features)
    # Adjust predictions to the number of classes
    #predictions_min, predictions_max = 0, 1
    #for i in range(len(predictions)):
    #    predictions[i] = int((predictions[i]-1e-6 - predictions_min) / (predictions_max - predictions_min) * N_CLASSES)  # discretise

    # 7.1) Accuracy
    accuracy = accuracy_score(test_targets, predictions)

    # 7.2) F1-Score
    f1 = f1_score(test_targets, predictions, average='weighted')

    print(f'Accuracy: {accuracy}')
    print(f'F1-Score: {f1}')
    print('-----\n')

    # 8. Plot Confusion Matrix
    from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
    import seaborn as sn
    import pandas as pd
    import matplotlib.pyplot as plt

    cm = confusion_matrix(test_targets, predictions)
    df_cm = pd.DataFrame(cm, range(N_CLASSES), range(N_CLASSES))
    plt.figure(figsize=(10, 7))
    sn.set(font_scale=1.4)  # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16})  # font size
    plt.show()

In [20]:
if HYPERPARAMETERS_TUNING and CV and not SELECTION and not SEQUENTIAL_SELECTION:
    print("Hyperparameter tuning")
    #_tunner = tunner(model, hyperparameters, resource='n_samples', min_resources=int(179*0.5),
    #                 cv=cv, scoring='f1_weighted', error_score=0, random_state=0,
    #                 verbose=2, n_jobs=-1)

    _tunner = tunner(model, hyperparameters,
                     cv=cv, scoring='f1_weighted', error_score=0, random_state=0,
                     verbose=2, n_jobs=-1)

    _tunner.fit(features[selected_features], features['targets'])
    print("Finished hyperparameter tuning")
    print("Best parameters:", _tunner.best_params_)
    print("Best score:", _tunner.best_score_)
    print("All results:")
    all_results = pd.DataFrame(_tunner.cv_results_)
    print(all_results)

In [21]:
if SEQUENTIAL_SELECTION and not SELECTION and not CV and not HYPERPARAMETERS_TUNING:

    def _train_test(model, experimental_features, train_size=0.85):
        train_dataset, test_dataset = train_test_split(features, train_size=train_size, shuffle=True, stratify=features['targets'], random_state=0)
        train_features, train_targets = train_dataset[experimental_features], train_dataset['targets']
        test_features, test_targets = test_dataset[experimental_features], test_dataset['targets']
        model.fit(train_features, train_targets)
        predictions = model.predict(test_features)
        f1 = f1_score(test_targets, predictions, average='weighted')
        return f1

    # Baseline test
    f1_baseline = _train_test(RandomForestClassifier(**params), selected_features)

    # Iterate through 'additional_features' and add the one that improves the most the model
    print("Sequential feature selection")
    results = {'baseline': f1_baseline}
    for feature in additional_features.columns:
        experimental_set_features = np.append(selected_features, feature)
        f1 = _train_test(RandomForestClassifier(**params), experimental_set_features, train_size=0.98)
        print(f"With '{feature}'\nF1-score = {f1}\n")
        results[feature] = f1